Importing libraries

In [550]:
import sqlite3
import pandas as pd
import pycytominer

In [551]:
inputDir = "/Users/llanos/Documents/GitHub/Kyra_project/Batch4"
database_name='DS2U_DS1_Neurons_03042024_database'

In [553]:
#READING Sqlite file


# Replace 'your_database.db' with the actual path to your SQLite database
database_path = f"{inputDir}/{database_name}.db"
#database_path = '/Users/llanos/Downloads/BR_109_68_D14_1.sqlite'

# Establish a connection to the SQLite database
connection = sqlite3.connect(database_path)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Query to retrieve table names from sqlite_master
table_query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query
cursor.execute(table_query)

# Add a new column named "Metadata_genotype" to the existing table
cursor.execute('''ALTER TABLE MyExpt_Per_Image
                  ADD COLUMN Metadata_genotype TEXT;''')

# Update the values in the new column based on the conditions

cursor.execute('''UPDATE MyExpt_Per_Image
                  SET Metadata_genotype = CASE 
                                       WHEN Image_Metadata_Well LIKE '%B%' OR Image_Metadata_Well LIKE '%C%' THEN 'DS2U'
                                       WHEN Image_Metadata_Well LIKE '%E%' OR Image_Metadata_Well LIKE '%F%' THEN 'DS1'
                                       ELSE NULL  -- You can change this to a default value if needed
                                       END;''')


# Fetch all table names
table_names = cursor.fetchall()


# Rename the table
old_table_Image = 'MyExpt_Per_Image'
new_table_Image = 'Image'
old_table_Nuclei = 'MyExpt_Per_Nuclei'
new_table_Nuclei = 'Nuclei'
old_table_Cytoplasm = 'MyExpt_Per_Cytoplasm'
new_table_Cytoplasm = 'Cytoplasm'
old_table_Cells = 'MyExpt_Per_Cells'
new_table_Cells = 'Cells'

# Use the ALTER TABLE statement to rename the table
cursor.execute(f"ALTER TABLE {old_table_Image} RENAME TO {new_table_Image}")
cursor.execute(f"ALTER TABLE {old_table_Nuclei} RENAME TO {new_table_Nuclei}")
cursor.execute(f"ALTER TABLE {old_table_Cytoplasm} RENAME TO {new_table_Cytoplasm}")
cursor.execute(f"ALTER TABLE {old_table_Cells} RENAME TO {new_table_Cells}")


#Add a new column for Image_Metadata_Table



# Step 1: Add the new column to the table
alter_query = '''
ALTER TABLE Image
ADD COLUMN Image_Metadata_Plate TEXT;
'''

cursor.execute(alter_query)

# Step 2: Update all existing rows in the table with the string "plate1"
update_query = '''
UPDATE Image
SET Image_Metadata_Plate = 'plate4';
'''

cursor.execute(update_query)


# Commit the changes
connection.commit()


# Close the cursor and database connection
cursor.close()
connection.close()



OperationalError: no such table: MyExpt_Per_Image

In [123]:
#Remove a column that was created previously. This is optional in case that you need to remove a column

import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect('/Users/llanos/Documents/GitHub/Kyra_project/Batch1/WC_NPC_database_subsetTables.db')  # Replace 'your_database.db' with your actual database file

# Create a cursor object
cursor = conn.cursor()

# Step 1: Remove the column from the table
alter_query = '''
ALTER TABLE Image
DROP COLUMN Image_Metadata_Plate;
'''

cursor.execute(alter_query)

# Commit the changes and close the connection
conn.commit()
conn.close()


Read the tables name in the database file

In [555]:
# Display the table names
for table_name in table_names:
    print(table_name[0])

In [556]:
#MAKE A NEW SQLITE FILE WITH THE FOUR TABLES THAT ARE READ FOR THE PROFILING RECEIPE

def copy_tables(source_db, destination_db, tables):
    with sqlite3.connect(source_db) as source_conn:
        with source_conn as source_cursor:
            # Attach the destination database
            source_cursor.execute(f"ATTACH DATABASE '{destination_db}' AS dest")

            # Loop through the specified tables and copy them to the destination
            for table in tables:
                source_cursor.execute(f"CREATE TABLE dest.{table} AS SELECT * FROM {table}")

# Replace these values with your actual database file names and table names
source_database = f"{inputDir}/{database_name}.db"
destination_database = f"{inputDir}/{database_name}.sqlite"
tables_to_copy = ['Cells', 'Nuclei', 'Cytoplasm','Image']

# Call the function to copy tables
copy_tables(source_database, destination_database, tables_to_copy)

In [126]:
#RENAME DATABASE O SQLITE FILE.

import os

# Specify the current and new file names
current_file_name = '/Users/llanos/Documents/GitHub/Kyra_project/Batch1/WC_NPC_database_subsetTables.db'
new_file_name = '/Users/llanos/Documents/GitHub/Kyra_project/Batch1/WC_NPC_database_subsetTables.sqlite'

# Rename the file
os.rename(current_file_name, new_file_name)

Convert .sqlite file in a dataframe. From CellProfiler we get a database as "Single object table" where all the object are in the same table

In the sqlite file you can see the different tables contained there.

Read the table that contain Objects: MyExpt_Per_Object

GROUPING EACH TABLE FROM SQLITE FILE BASED ON IMAGENUMBER COLUMN

In [557]:

connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Cells"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Cells.csv", index=False) 


#data_frame.columns

grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_Cells = grouped_df.mean().reset_index(drop=False)
result_df_Cells.shape



(543, 374)

In [558]:
connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Nuclei"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Nuclei.csv", index=False) 
data_frame_objects.head()
data_frame_objects.shape
grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_Nuclei = grouped_df.mean().reset_index(drop=False)
result_df_Nuclei.shape


(543, 374)

In [559]:
connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Cytoplasm"
data_frame_objects = pd.read_sql(query, connection)
data_frame_objects.to_csv(f"{inputDir}/Cytoplasm.csv", index=False) 

#data_frame.columns

grouped_df = data_frame_objects.groupby('ImageNumber')

# Applying aggregate functions (e.g., sum, mean) to each group
result_df_cytoplasm = grouped_df.mean().reset_index(drop=False)

result_df_cytoplasm.shape


(543, 188)

In [560]:
connection = sqlite3.connect(f"{inputDir}/{database_name}.db")
query = "SELECT * FROM Image"
data_frame_image = pd.read_sql(query, connection)

data_frame_image.to_csv(f"{inputDir}/Image.csv", index=False) 
data_frame_image.head()
data_frame_image.shape
#data_frame.columns

(659, 443)

In [561]:
# Merge the DataFrames using 'ID'
merged_df = pd.merge(result_df_Cells, result_df_Nuclei, on='ImageNumber', how='inner')
merged_df = pd.merge(merged_df, result_df_cytoplasm, on='ImageNumber', how='inner')
merged_df = pd.merge(merged_df, data_frame_image, on='ImageNumber', how='inner')
merged_df.head()
print(merged_df.shape)

(543, 1376)


In [562]:
# Check for empty columns and count them
empty_columns = merged_df.columns[merged_df.isnull().all()]
num_empty_columns = len(empty_columns)

# Display the result
if num_empty_columns == 0:
    print("No empty columns.")
else:
    print(f"There are {num_empty_columns} empty columns:")
    print(empty_columns)

There are 266 empty columns:
Index(['Image_Intensity_LowerQuartileIntensity_Brightfield_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Mtcndr_sgmntd',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Nuclei',
       'Image_Intensity_LowerQuartileIntensity_Brightfield_Nuclel_sgmntd',
       'Image_Intensity_LowerQuartileIntensity_DNA_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_DNA_Mitocondria_segmented',
       'Image_Intensity_LowerQuartileIntensity_DNA_Nuclei',
       'Image_Intensity_LowerQuartileIntensity_DNA_Nucleoli_segmented',
       'Image_Intensity_LowerQuartileIntensity_ER_Cytoplasm',
       'Image_Intensity_LowerQuartileIntensity_ER_Mitocondria_segmented',
       ...
       'Image_Intensity_UpperQuartileIntensity_Mito_Cytoplasm',
       'Image_Intensity_UpperQuartileIntensity_Mito_Mitocondria_segmentd',
       'Image_Intensity_UpperQuartileIntensity_Mito_Nuclei',
       'Image_Intensity_UpperQuartileIntensity_Mito_Nucleoli_segm

In [563]:
# drop empty columns
merged_df_dropped_empty = merged_df.drop(empty_columns, axis=1)
merged_df_dropped_empty.shape

(543, 1110)

In [564]:
# Rename the first column to include the label Metadara
sorted_df_rename = merged_df_dropped_empty.rename(columns={'Image_ImageNumber': 'Image_Metadata_ImageNumber'})

In [565]:
columns = merged_df_dropped_empty.columns

# Sort column names
sorted_columns = sorted(columns, key=lambda x: x.startswith("Image_Metadata"), reverse=True)

# Create a new DataFrame with sorted columns
sorted_df = merged_df_dropped_empty[sorted_columns]
sorted_df.head()

,Image_Metadata_DiseaseGroup,Image_Metadata_Frame,Image_Metadata_Series,Image_Metadata_Site,Image_Metadata_Well,Image_Metadata_WellColumn,Image_Metadata_WellRow,Image_Metadata_Plate,ImageNumber,Cells_Number_Object_Number,...,Image_URL_Membrane,Image_URL_Mito,Image_URL_RNA_nucleoli,Image_Width_Brightfield,Image_Width_DNA,Image_Width_ER,Image_Width_Membrane,Image_Width_Mito,Image_Width_RNA_nucleoli,Metadata_genotype
0,DS2U_DS1 iN,0,0,01,B02,02,02,plate4,1,2.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
1,DS2U_DS1 iN,0,0,02,B02,02,02,plate4,2,5.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
2,DS2U_DS1 iN,0,0,07,B02,02,02,plate4,7,1.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
3,DS2U_DS1 iN,0,0,10,B02,02,02,plate4,10,9.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U
4,DS2U_DS1 iN,0,0,12,B02,02,02,plate4,12,1.0,...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,file:///Z:/Kyra_Export_PB/USE_THESIS%20iN%20DS...,1080,1080,1080,1080,1080,1080,DS2U


In [566]:
sorted_df.to_csv(f"{inputDir}/sorted_df.csv", index=False) 

In [567]:
sorted_df.shape

(543, 1110)

In [568]:
#delete the Image prefix in the columns.
sorted_df.columns = [col.replace('Image_Metadata', 'Metadata') if 'Image_Metadata' in col else col for col in sorted_df.columns]
sorted_df.to_csv(f"{inputDir}/sorted_df.csv", index=False) 

In [569]:
# Extract column names with 'Metadata'
# Extract column names with 'Metadata'
metadata_columns = [col for col in sorted_df.columns if 'Metadata' in col]
print(len(metadata_columns))

# Extract column names with 'Cells', 'Nuclei', or 'Cytoplasm'
cell_related_columns = [col for col in sorted_df.columns if any(keyword in col for keyword in ['Cells', 'Nuclei', 'Cytoplasm'])]
print(len(cell_related_columns))



10
1010


In [570]:

# Extract column names with 'Image' prefix, excluding those already in cell_related_columns
image_columns = [col for col in sorted_df.columns if col.startswith('Image') and col not in cell_related_columns and col not in metadata_columns]


# Determine the remaining columns
other_columns = [col for col in sorted_df.columns if col not in metadata_columns + cell_related_columns + image_columns]


# Concatenate the column lists in the desired order
sorted_columns_final = metadata_columns + other_columns + cell_related_columns + image_columns


# Create a new DataFrame with the sorted columns
sorted_df_final = sorted_df[sorted_columns_final]

In [571]:
# Explicitly convert all non-metadata columns to float
# in the original df ALL columns are of type object and that breaks pycytominer
for col in sorted_df_final.columns:
    if 'Nuclei' in col or 'Cells' in col or 'Cytoplasm' in col or 'ImageNumber' in col:
        sorted_df_final[col] = sorted_df_final[col].astype(float)
#print([df.dtypes])

sorted_df_final.to_csv(f"{inputDir}/sorted_df_final.csv", index=False)


In [572]:
duplicated_columns = sorted_df_final[sorted_df_final.duplicated()]

if len(duplicated_columns) > 0:
    print(f"Duplicate columns found: {', '.join(duplicated_columns)}")
else:
    print("No duplicate columns found.")

No duplicate columns found.


In [573]:
# Define the substrings to replace
substrings_to_replace = ['_Nuclei']
replacement_substrings = ['_Nucle']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [575]:
# Define the substrings to replace
substrings_to_replace = ['_Cells']
replacement_substrings = ['_Cell']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [576]:
# Define the substrings to replace
substrings_to_replace = ['_Cytoplasm']
replacement_substrings = ['_Cytoplas']

# Replace substrings in column names
sorted_df_final.columns = [col.replace(old, new) for col in sorted_df_final.columns for old, new in zip(substrings_to_replace, replacement_substrings)]


In [577]:
# Filter columns that contain "Nuclei," "Cytoplasm," and "Cells"
filtered_df = sorted_df_final.filter(like='Nuclei').join(sorted_df_final.filter(like='Cytoplasm')).join(sorted_df_final.filter(like='Cells'))
filtered_df.to_csv(f"{inputDir}/filtered_df.csv", index=False)

In [578]:
#Save the columns discarded in other dataframe


# Get column names that start with "Metadata_"
metadata_column_names = [col for col in sorted_df_final.columns if col.startswith('Metadata_')]

# Create a new DataFrame with the filtered columns
Columns_metadata = sorted_df_final[metadata_column_names]
Columns_metadata.to_csv(f"{inputDir}/Columns_metadata.csv", index=False)


In [579]:
# Define the aggregation functions
aggregations = {
    'Metadata_Well': 'first',  # Take the first value as it's the same for each group
    **{col: ['mean', 'median', 'std', 'min', 'max'] for col in filtered_df.columns}
}
# Group by "Metadata_ImageNumber" and aggregate the statistics for columns with the specified prefix
# The .agg (aggregate) method applies the listed operations to all values grouped by ImageNumber
well_agg_df = sorted_df_final.groupby(['Metadata_Well','Metadata_genotype'], sort=False).agg(aggregations)

In [580]:
# Flatten MultiIndex columns
well_agg_df.columns = well_agg_df.columns.map('_'.join)

# Reset index to move 'Metadata_ImageNumber' to a regular column
well_agg_df.reset_index(inplace=True)

/var/folders/mm/mp46tk592c10r7w5n0vwcclc0000gq/T/ipykernel_64893/292143542.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  well_agg_df.reset_index(inplace=True)
/var/folders/mm/mp46tk592c10r7w5n0vwcclc0000gq/T/ipykernel_64893/292143542.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  well_agg_df.reset_index(inplace=True)


In [581]:
well_agg_df.head()
well_agg_df.to_csv(f"{inputDir}/well_agg_df.csv", index=False)

In [598]:
well_agg_df = pd.read_csv("/Users/llanos/Documents/GitHub/Kyra_project/Batch3/well_agg_df.csv")

In [599]:
pycytominer.normalize(
    profiles=well_agg_df,    #the dataframe containing the CellProfiler features
    #features=feature_cols,   #list of features to normalize; if "infer", anything starting with 'Cells', 'Nuclei' or 'Cytoplasm'
    #meta_features=metadata_cols,   #list of metadata columns; if "infer" (default), any column starting with 'Metadata_'
    image_features=False,  #Whether the profiles contain image features
    samples="Metadata_genotype=='WCB'",     #The metadata column values to use as a normalization reference. Defaults to "all".
    method="mad_robustize",    #How to normalize the dataframe. Defaults to "standardize". "mad_robustize" is used in the profiling recipe by default
    mad_robustize_epsilon=0, # The mad_robustize fudge factor parameter. Set this to 0 if mad_robustize generates features with large values.
    output_file=f"{inputDir}/data_normalized_WC.csv" 
)
df_norm = pd.read_csv(f"{inputDir}/data_normalized_WC.csv")
df_norm.shape

(20, 4663)

CONCATENATE BEFORE FUTURE SELECT:



In [600]:
df_norm_DS = pd.read_csv(f"{inputDir}/data_normalized.csv")


In [602]:
df_norm_DS.shape

(20, 4663)

In [603]:
df_norm.shape

(20, 4663)

In [604]:
#Concatenate plates

df_norm_WC_DS = pd.concat([df_norm, df_norm_DS], ignore_index=True)

In [605]:
df_norm_WC_DS.shape

(40, 4663)

In [606]:
pycytominer.feature_select(
    profiles=df_norm_WC_DS,
    #features=feature_cols,   #list of features to normalize; if "infer", anything starting with 'Cells', 'Nuclei' or 'Cytoplasm'
    image_features=False,  #Whether the profiles contain image features
    operation=['variance_threshold','correlation_threshold', 'drop_na_columns', 'blocklist', 'drop_outliers'],   #add outlier removal to get rid of 
    outlier_cutoff=500, #500 is the default
    output_file=f"{inputDir}/data_normalized_featureSelected_WC_DS_bygenotype.csv", 
    samples='all', #Samples to provide operation on.Str or list. Defaults to 'all' ...I don't get it
)
df_ftsel_WC_DS = pd.read_csv(f"{inputDir}/data_normalized_featureSelected_WC_DS_bygenotype.csv")
df_ftsel_WC_DS.shape

(40, 1439)

In [607]:
df_ftsel_WC_DS.shape
df_ftsel_WC_DS.to_csv(f"{inputDir}/df_ftsel_WC_DS.csv", index=False)


In [608]:
# Reorder and transpose df for easier import into Morpheus
df_ftsel_WC_DS = df_ftsel_WC_DS.T

#save to final .csv
# Index=True is necessary so that we save the row labels into the .csv
df_ftsel_WC_DS.to_csv(f"{inputDir}/data_forMorpheus_WC_DS_by_genotype.csv",sep=',',encoding="utf-8",header=False,index=True)

This table as a ID contain the ImageNumber

Read the table that contain Image data: MyExpt_Per_Image

In [5]:
connection = sqlite3.connect(database_path)
query = "SELECT * FROM MyExpt_Per_Image"
data_frame_image = pd.read_sql(query, connection)
#data_frame.columns
data_frame_image.to_csv('/Users/llanos/Documents/GitHub/Kyra_project/Subset_PR/Subset_MyExpt_Per_Image.csv', index=False) 



In [50]:
data_frame_image.head()

,ImageNumber,Image_Count_Cells,Image_Count_CellsPR_Related,Image_Count_CellsPR_filtered,Image_Count_Cells_PR1,Image_Count_Cytoplasm,Image_Count_Mitocondria_related,Image_Count_Mitocondria_segmented,Image_Count_Neurons,Image_Count_Neurons1,...,Image_URL_ER,Image_URL_Membrane,Image_URL_Mito,Image_URL_RNA_nucleoli,Image_Width_Brightfield,Image_Width_DNA,Image_Width_ER,Image_Width_Membrane,Image_Width_Mito,Image_Width_RNA_nucleoli
0,1,11,11,11,11,11,3,3,0,0,...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,1080,1080,1080,1080,1080,1080
1,2,14,14,14,14,14,72,72,0,0,...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,1080,1080,1080,1080,1080,1080
2,3,3,2,2,2,3,0,0,1,1,...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,1080,1080,1080,1080,1080,1080
3,4,14,13,13,13,14,10,10,1,1,...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,1080,1080,1080,1080,1080,1080
4,5,15,14,14,14,15,19,19,1,1,...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,file:/Users/llanos/Documents/Kyra_images/DS2U%...,1080,1080,1080,1080,1080,1080


In [7]:
print(data_frame_image.shape)
print(data_frame_objects.shape)


(6, 395)
(133, 1172)


Get the disease group and add as a column to the dataframe

In [18]:


# Extract the desired part of the path and create a new column
data_frame_image['Image_Metadata_Disease'] = data_frame_image['Image_PathName_Brightfield'].str.extract(r'\\([\w\s-]+)\\Images')


This table contain ImageNumber and Well as ID

In [19]:
annotated_features = pd.merge(data_frame_image,data_frame_objects, on='ImageNumber', how='outer')
annotated_features.head()

#annotated_features = aggregated_features.merge(stripped_image_metadata, left_on=['Image_Metadata_Plate','Image_Metadata_Well'], right_on=['Image_Metadata_Plate_DAPI','Image_Metadata_Well_DAPI'], how='left')
#annotated_features.columns = annotated_features.columns.str.replace(r'^Image_Metadata', 'Metadata', regex=True)
#Replace "Image in columns headers by "Metadata"
annotated_features.rename(columns=lambda x: x.replace('Image', 'Metadata'), inplace=True)




In [55]:
#the number of columns is the sum between the two dataframe, the only column that was merged was ImageNumber
annotated_features.shape

(133, 1567)

In [56]:
annotated_features.to_csv('Subset_Neurons_database.csv', index=False) 

In [57]:
annotated_features.shape

(133, 1567)

In [40]:
#SANITY CHECK
unique_values_count = annotated_features['Metadata_Metadata_Well'].nunique()
print(unique_values_count)


# Check for completely empty columns
empty_columns = annotated_features.columns[annotated_features.isnull().all()]

# Count the number of completely empty columns
num_empty_columns = len(empty_columns)
print("Number of completely empty columns:", num_empty_columns)
duplicated_columns = annotated_features[annotated_features.duplicated()]

# Count the occurrences of each duplicated column name
column_duplicates_count = annotated_features.columns.duplicated().sum()

#print("Duplicated columns:", duplicated_columns.tolist())
print("Number of duplicated columns:", column_duplicates_count)


1
Number of completely empty columns: 215
Number of duplicated columns: 0


In [58]:
annotated_features.shape
# Select all numerical columns, including integers and floats
numerical_metadata_cols = annotated_features.select_dtypes(include='number').columns.difference(['Metadata_Metadata_Well'])

# Drop selected columns
#numerical_df = annotated_features.drop(columns=numerical_metadata_cols)

In [60]:
# Using groupby and aggregate functions
agg_funcs = {col: 'mean' for col in numerical_metadata_cols}
aggregate = annotated_features.groupby('Metadata_Metadata_Well',as_index=False).agg({**agg_funcs, 'Metadata_Metadata_Well': 'first'}).reset_index()
#aggregate_df = aggregate.mean().reset_index()

In [61]:
aggregate.head(50)

,index,CellsPR_Related_Location_Center_X,CellsPR_Related_Location_Center_Y,CellsPR_Related_Location_Center_Z,CellsPR_Related_Number_Object_Number,CellsPR_Related_Parent_CellsPR_filtered,CellsPR_filtered_AreaShape_Area,CellsPR_filtered_AreaShape_BoundingBoxArea,CellsPR_filtered_AreaShape_BoundingBoxMaximum_X,CellsPR_filtered_AreaShape_BoundingBoxMaximum_Y,...,Nucleoli_segmented_Location_MaxIntensity_Y_RNA_nucleoli,Nucleoli_segmented_Location_MaxIntensity_Z_DNA,Nucleoli_segmented_Location_MaxIntensity_Z_ER,Nucleoli_segmented_Location_MaxIntensity_Z_Membrane,Nucleoli_segmented_Location_MaxIntensity_Z_Mito,Nucleoli_segmented_Location_MaxIntensity_Z_RNA_nucleoli,Nucleoli_segmented_Number_Object_Number,Nucleoli_segmented_Parent_Nuclei,ObjectNumber,Metadata_Metadata_Well
0,0,605.645825,453.024636,0.0,6.745455,6.745455,4153.509091,15600.236364,649.581818,502.0,...,454.556962,0.0,0.0,0.0,0.0,0.0,12.139241,7.35443,24.578947,F02


In [63]:



print(aggregate.shape)

# Get the names of dropped columns
dropped_columns = list(set(annotated_features.columns) - set(aggregate.columns))

print("Dropped columns:", dropped_columns)

(1, 1323)
Dropped columns: ['Metadata_Intensity_StdIntensity_DNA_Nuclei', 'Metadata_Intensity_MeanIntensity_RNA_nucleoli_Cytoplasm', 'Metadata_Intensity_StdIntensity_Membrane_Nuclei', 'Metadata_Intensity_MaxIntensity_Membrane_Cytoplasm', 'Metadata_Intensity_StdIntensity_Mito_Nuclei', 'Metadata_Intensity_MinIntensity_Membrane_Nuclei', 'Metadata_Intensity_MeanIntensity_Membrane_Nuclei', 'Metadata_Intensity_MaxIntensity_Brightfield_Nuclei', 'Metadata_PathName_ER', 'Metadata_Intensity_TotalIntensity_RNA_nucleoli_Neurons_Related', 'Metadata_Intensity_TotalIntensity_ER_Cytoplasm', 'Metadata_Intensity_LowerQuartileIntensity_Brightfield_Nuclei', 'Metadata_Intensity_TotalIntensity_RNA_nucleoli_Cytoplasm', 'Metadata_Intensity_UpperQuartileIntensity_ER_Neurons_Related', 'Metadata_Intensity_MedianIntensity_ER_Cytoplasm', 'Metadata_URL_ER', 'Neurons2_Number_Object_Number', 'Metadata_Intensity_MaxIntensity_DNA_Neurons_Related', 'Metadata_MD5Digest_Brightfield', 'Metadata_Intensity_TotalIntensity_Mem

In [64]:
#Generating Backend file

#add column "Metadata_Plate"
aggregate.insert(0, 'Metadata_Plate', 'plate1')


In [65]:
#SAVING THE FILE: THIS IS THE BACKEND FILE
aggregate.to_csv('/Users/llanos/Documents/GitHub/Kyra_project/Subset_PR/Subset_Neurons_database_aggregated.csv', index=False) 

In [64]:
metadata_cols = [x for x in aggregate_df.columns if 'Metadata' in x]
print(metadata_cols)
non_metadata_cols = [x for x in  aggregate_df.columns if 'Metadata' not in x]
print(non_metadata_cols)
reordered_annotated_features =  aggregate_df[metadata_cols+non_metadata_cols]
namefixed_annotated_columns = {x:x[x.index('Metadata_'):] for x in reordered_annotated_features.columns if 'Metadata_' in x}
namefixed_annotated_features = reordered_annotated_features.rename(columns=namefixed_annotated_columns)
#columns_to_manually_remove = ['Metadata_Plate_DAPI','Metadata_Well_DAPI','Metadata_compound',
#                              'Metadata_concentration','TableNumber','ImageNumber','Image_FileName_DAPI',
#                              'Image_PathName_DAPI','Image_FileName_Tubulin','Image_PathName_Tubulin',
#                              'Image_FileName_Actin','Image_PathName_Actin','Replicate']
#cleaned_features = namefixed_annotated_features.drop(columns=columns_to_manually_remove)
#cleaned_features.shape
#cleaned_features.head()

['Image_Metadata_Well', 'Image_ExecutionTime_02Metadata']
['index', 'ImageNumber', 'ObjectNumber', 'Cells_Number_Object_Number', 'Cells_AreaShape_Area', 'Cells_AreaShape_BoundingBoxArea', 'Cells_AreaShape_BoundingBoxMaximum_X', 'Cells_AreaShape_BoundingBoxMaximum_Y', 'Cells_AreaShape_BoundingBoxMinimum_X', 'Cells_AreaShape_BoundingBoxMinimum_Y', 'Cells_AreaShape_Center_X', 'Cells_AreaShape_Center_Y', 'Cells_AreaShape_CentralMoment_0_0', 'Cells_AreaShape_CentralMoment_0_1', 'Cells_AreaShape_CentralMoment_0_2', 'Cells_AreaShape_CentralMoment_0_3', 'Cells_AreaShape_CentralMoment_1_0', 'Cells_AreaShape_CentralMoment_1_1', 'Cells_AreaShape_CentralMoment_1_2', 'Cells_AreaShape_CentralMoment_1_3', 'Cells_AreaShape_CentralMoment_2_0', 'Cells_AreaShape_CentralMoment_2_1', 'Cells_AreaShape_CentralMoment_2_2', 'Cells_AreaShape_CentralMoment_2_3', 'Cells_AreaShape_Compactness', 'Cells_AreaShape_ConvexArea', 'Cells_AreaShape_Eccentricity', 'Cells_AreaShape_EquivalentDiameter', 'Cells_AreaShape_Eule

In [27]:
namefixed_annotated_features.head()

,Metadata_Well,Image_ExecutionTime_02Metadata,index,ImageNumber,ObjectNumber,Cells_Number_Object_Number,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,...,Image_Threshold_WeightedVariance_Cells,Image_Threshold_WeightedVariance_Mitocondria_segmented,Image_Threshold_WeightedVariance_Nuclei,Image_Threshold_WeightedVariance_Nucleoli_segmented,Image_Width_Brightfield,Image_Width_DNA,Image_Width_ER,Image_Width_Membrane,Image_Width_Mito,Image_Width_RNA_nucleoli
0,B02,0.0,0,3.502374,461.470123,184.017195,1058.537346,2654.990865,543.308436,590.518001,...,0.280613,0.452426,0.261507,0.426685,1080.0,1080.0,1080.0,1080.0,1080.0,1080.0
1,B04,0.0,1,7.859091,198.735227,58.738397,1608.232068,4628.383966,689.063291,575.514768,...,0.200269,0.493772,0.145162,0.530312,1080.0,1080.0,1080.0,1080.0,1080.0,1080.0


In [111]:
normalized = pycytominer.normalize(profiles=namefixed_annotated_features ,method='mad_robustize')

normalized.to_csv('neurons_normalized_mad_robustize.csv',index=False)

pycytominer.cyto_utils.write_gct(normalized,output_file='Neurons_normalized_mad_robustize.gct')

normalized.head()

,Metadata_Well,Cells_Number_Object_Number,Cells_AreaShape_Area,Cells_AreaShape_BoundingBoxArea,Cells_AreaShape_BoundingBoxMaximum_X,Cells_AreaShape_BoundingBoxMaximum_Y,Cells_AreaShape_BoundingBoxMinimum_X,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_Center_X,Cells_AreaShape_Center_Y,...,Nuclei_AreaShape_Zernike_9_3,Nuclei_AreaShape_Zernike_9_5,Nuclei_AreaShape_Zernike_9_7,Nuclei_AreaShape_Zernike_9_9,Nuclei_Children_Cells_Count,Nuclei_Children_Cytoplasm_Count,Nuclei_Location_Center_X,Nuclei_Location_Center_Y,Nuclei_Location_Center_Z,Nuclei_Math_area_ratio
0,F02,1.492978,-1.266293,-1.086834,-0.519542,0.711076,-0.615763,0.685927,-0.587913,0.678770,...,1.883021,-0.195625,0.238913,0.365038,0.0,0.0,-0.606109,0.704737,0.0,0.603732
1,F03,0.741663,-0.138062,-0.010371,0.356315,-0.166569,0.070639,-0.102473,0.267228,-0.119533,...,0.564382,0.550784,1.039415,0.725554,0.0,0.0,0.275334,-0.147610,0.0,0.022784
2,F04,0.970039,-0.221474,0.145533,-0.078048,0.149857,-0.322601,0.102473,-0.176224,0.119533,...,-1.486497,-0.617605,-0.304691,0.055284,0.0,0.0,-0.164321,0.147610,0.0,0.528982
3,F05,0.242465,-0.271435,0.010371,1.279740,3.221580,1.068943,2.570646,1.318077,2.886540,...,0.542385,-1.325067,0.252864,1.476376,0.0,0.0,1.300947,2.886608,0.0,0.231416
4,F06,0.039803,0.249752,0.462540,-1.395660,1.608360,-1.672645,1.129040,-1.654168,1.348424,...,1.727142,0.585978,0.740498,-0.055284,0.0,0.0,-1.659722,1.383784,0.0,-0.667986


In [112]:
feature_selected = pycytominer.feature_select(profiles=normalized, operation = ['variance_threshold','correlation_threshold','drop_na_columns','blocklist'])

feature_selected.to_csv('Neurons_feature_selected_mad_robustize.csv',index=False)

pycytominer.cyto_utils.write_gct(feature_selected,output_file='Neurons_feature_selected_mad_robustize.gct')

In [113]:
feature_selected.head()

,Metadata_Well,Cells_AreaShape_BoundingBoxMinimum_Y,Cells_AreaShape_CentralMoment_0_1,Cells_AreaShape_CentralMoment_1_1,Cells_AreaShape_CentralMoment_1_3,Cells_AreaShape_CentralMoment_2_1,Cells_AreaShape_CentralMoment_2_2,Cells_AreaShape_Compactness,Cells_AreaShape_Eccentricity,Cells_AreaShape_HuMoment_4,...,Nuclei_AreaShape_Zernike_8_0,Nuclei_AreaShape_Zernike_8_2,Nuclei_AreaShape_Zernike_8_4,Nuclei_AreaShape_Zernike_8_6,Nuclei_AreaShape_Zernike_8_8,Nuclei_AreaShape_Zernike_9_1,Nuclei_AreaShape_Zernike_9_3,Nuclei_AreaShape_Zernike_9_5,Nuclei_AreaShape_Zernike_9_7,Nuclei_AreaShape_Zernike_9_9
0,F02,0.685927,1.120593,-0.045502,0.193021,0.645765,-0.711149,-1.275917,-2.408876,5.600438,...,3.292498,-0.008313,0.985063,0.802632,0.079859,-0.374904,1.883021,-0.195625,0.238913,0.365038
1,F03,-0.102473,-0.007173,-0.001898,0.074149,-0.049469,-0.430391,0.400625,0.900894,-0.564364,...,-0.592504,0.008313,0.138216,-2.215304,-0.252438,-0.375477,0.564382,0.550784,1.039415,0.725554
2,F04,0.102473,-0.400288,0.311445,0.140206,1.249388,-0.041925,0.090049,0.786194,0.192704,...,-2.071394,2.153284,0.796938,-0.076434,0.037650,0.882804,-1.486497,-0.617605,-0.304691,0.055284
3,F05,2.570646,-0.352749,-1.219448,-2.639847,-1.652579,2.273288,-0.342531,0.719103,0.925258,...,-0.145709,-0.694099,-0.298596,0.647733,0.103706,-0.685100,0.542385,-1.325067,0.252864,1.476376
4,F06,1.129040,0.717468,-0.911519,-1.466574,0.834800,1.522981,0.454199,0.315210,212.196217,...,0.503128,0.807675,0.496614,0.739012,1.634283,0.663882,1.727142,0.585978,0.740498,-0.055284


In [114]:
feature_selected.shape

(20, 166)

In [ ]:
name: profiling
dependencies:
- conda-forge::python=3.7.1
- conda-forge::pandas=1.2.0
- conda-forge::pip=19.2.2
- conda-forge::pyyaml=5.3.1
- conda-forge::plotly=4.14.3
- conda-forge::dvc-s3
- plotly::python-kaleido=0.0.3
- pip
- pip:
  - git+https://github.com/cytomining/pycytominer@36241269c4293c24484986568ca16b2d7eb9e808

In [ ]:
DATA='Progenitor_project'
PROJECT_NAME='Progenitor_project'
CONFIG_FILE="config_batch1" 

#creating the environment

 conda create -n profiling python=3.9 pyyaml plotly pip 'pandas<2'

pipeline: analysis
output_dir: profiles
platemap_well_column: Metadata_well_position
compartments:
  - cells
  - cytoplasm
  - nuclei
  - Mitocondria
  - Nucleoli
aggregate:
  perform: false
  plate_column: Metadata_Plate
  well_column: Metadata_Well
  method: median
  fields: all
  image_feature_categories:
    - Count
    - Intensity
annotate:
  perform: true
  well_column: Metadata_Well
  external :
    perform: false
    file: <metadata file name>
    merge_column: <Column to merge on>
normalize:
  perform: true
  method: mad_robustize
  features: infer
  mad_robustize_fudge_factor: 0
  image_features: true
  min_cells: 1
normalize_negcon:
  perform: false
  method: mad_robustize
  features: infer
  mad_robustize_fudge_factor: 0
  image_features: true
  min_cells: 1
feature_select:
  perform: true
  features: infer
  level: batch
  gct: true
  image_features: true
  operations:
    - variance_threshold
    - correlation_threshold
    - drop_na_columns
    - blocklist
  min_cells: 1
feature_select_negcon:
  perform: false
  features: infer
  level: batch
  gct: false
  image_features: true
  operations:
    - variance_threshold
    - correlation_threshold
    - drop_na_columns 
    - blocklist
  min_cells: 1 
quality_control:
  perform: false 
  summary:
    perform: true
    row: Metadata_Row 
    column: Metadata_Col
  heatmap:
    perform : true
options:
  compression: gzip
  float_format: "%.5g"
  samples: all
---
batch: batch1
plates:
  - name: plate1
    process: true
process: true


BATCH="batch1"
mkdir -p profiles/${BATCH}

#run the backend 

mkdir -p ~/ebs_tmp/${PROJECT_NAME}/workspace/scratch/${BATCH_ID}/
PLATES=$(readlink -f ~/ebs_tmp/${PROJECT_NAME}/workspace/scratch/${BATCH_ID}/plates_to_process.txt)
aws s3 ls s3://${BUCKET}/projects/${PROJECT_NAME}/workspace/analysis/${BATCH_ID}/ |cut -d " " -f29 | cut -d "/" -f1 >> ${PLATES}


cd ~/ebs_tmp/${PROJECT_NAME}/workspace/software/pycytominer/
mkdir -p  ../../log/BR00140145_3/
parallel \
--max-procs ${MAXPROCS} \
--ungroup \
--eta \
--joblog ../../log/BR00140145_3/collate.log \
--results ../../log/BR00140145_3/collate \
--files \
--keep-order \
python3 pycytominer/cyto_utils/collate_cmd.py ${BATCH_ID} pycytominer/cyto_utils/database_config/ingest_config.ini plate1 \
--tmp-dir ~/ebs_tmp \
--aggregate-only \
--dont-add-image-features \

python3 pycytominer/cyto_utils/collate_cmd.py batch1 --aggregate-only pycytominer/cyto_utils/database_config/ingest_config.ini ${BATCH_ID} \

python3 pycytominer/cyto_utils/collate_cmd.py batch1 pycytominer/cyto_utils/database_config/ingest_config.ini plate1
